In [14]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from datetime import datetime

from omegaconf import OmegaConf, DictConfig
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

from torch.optim import SGD, Adam, AdamW
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from adamp import AdamP

from albumentations import *
from albumentations.pytorch import ToTensorV2

now = datetime.now()
now = now.strftime("%y%m%d/%H%M%S")
%config Completer.use_jedi = False

In [15]:
s = \
'''
seed: 42
device_list: [0]
data_dir: /opt/ml/input/data/train
test_dir: /opt/ml/input/data/eval

train:
  n_fold: 5
  use_fold: [0]
  n_epochs: 20
  
dataset:
  image_size: [320, 256]
  batch_size: 128
  num_workers: 0

network:
  model_name: efficientnet-b0
  optimizer: AdamP  # ['SGD', 'Adam', 'AdamW']
  learning_rate: 3e-4

run_test: true
debug: false
tb: true
'''
conf = OmegaConf.create(s)
conf.log_dir = f'/opt/ml/logs/{now}'  # 학습 설정 및 결과는 이 경로에 저장됩니다.

if not os.path.isdir(conf.log_dir):
    os.makedirs(conf.log_dir, exist_ok=True)
OmegaConf.save(conf, os.path.join(conf.log_dir, 'config.yaml'))

if conf.debug:
    conf.train.n_epochs = 2
    conf.dataset.batch_size = 4
    conf.tb = False

print(OmegaConf.to_yaml(conf))

seed: 42
device_list:
- 0
data_dir: /opt/ml/input/data/train
test_dir: /opt/ml/input/data/eval
train:
  n_fold: 5
  use_fold:
  - 0
  n_epochs: 20
dataset:
  image_size:
  - 320
  - 256
  batch_size: 128
  num_workers: 0
network:
  model_name: efficientnet-b0
  optimizer: AdamP
  learning_rate: 0.0003
run_test: true
debug: false
tb: true
log_dir: /opt/ml/logs/210408/070814



In [16]:
def get_mask_label(image_name):
    """
    이미지 파일 이름을 통해 mask label을 구합니다.

    :param image_name: 학습 이미지 파일 이름
    :return: mask label
    """
    if 'incorrect_mask' in image_name:
        return 1
    elif 'normal' in image_name:
        return 2
    elif 'mask' in image_name:
        return 0
    else:
        raise ValueError(f'No class for {image_name}')


def get_gender_label(gender):
    """
    gender label을 구하는 함수입니다.
    :param gender: `male` or `female`
    :return: gender label
    """
    return 0 if gender == 'male' else 1


def get_age_label(age):
    """
    age label을 구하는 함수입니다.
    :param age: 나이를 나타내는 int.
    :return: age label
    """
    return 0 if int(age) < 30 else 1 if int(age) < 58 else 2

def convert_gender_age(gender, age):
    """
    gender와 age label을 조합하여 고유한 레이블을 만듭니다.
    이를 구하는 이유는 train/val의 성별 및 연령 분포를 맞추기 위함입니다. (by Stratified K-Fold)
    :param gender: `male` or `female`
    :param age: 나이를 나타내는 int.
    :return: gender & age label을 조합한 레이블
    """
    gender_label = get_gender_label(gender)
    age_label = get_age_label(age)
    return gender_label * 3 + age_label


def convert_label(image_path, sep=False):
    """
    이미지의 label을 구하는 함수입니다.
    :param image_path: 이미지 경로를 나타내는 str
    :param sep: 마스크, 성별, 연령 label을 따로 반환할건지 합쳐서 할지 나타내는 bool 인수입니다. 참일 경우 따로 반환합니다.
    :return: 이미지의 label (int or list)
    """
    image_name = image_path.split('/')[-1]
    mask_label = get_mask_label(image_name)

    profile = image_path.split('/')[-2]
    image_id, gender, race, age = profile.split("_")
    gender_label = get_gender_label(gender)
    age_label = get_age_label(age)
    if sep:
        return mask_label, gender_label, age_label
    else:
        return mask_label * 6 + gender_label * 3 + age_label

In [17]:
mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)


def get_transforms(need=('train', 'val'), img_size=(512, 384)):
    """
    Augmentation 함수를 반환합니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            CenterCrop(448, 336, p=1.0),
            RandomResizedCrop(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.3),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.3),
#             RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.3),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.3),
            Cutout(p=0.3),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            CenterCrop(448, 336, p=1.0),
            Resize(img_size[0], img_size[1], p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [18]:
IMG_EXTENSIONS = [
    ".jpg", ".JPG", ".jpeg", ".JPEG", ".png",
    ".PNG", ".ppm", ".PPM", ".bmp", ".BMP",
]


def is_image_file(filepath):
    """
    해당 파일이 이미지 파일인지 확인합니다.
    """
    return any(filepath.endswith(extension) for extension in IMG_EXTENSIONS)


def remove_hidden_file(filepath):
    """
    `._`로 시작하는 숨김 파일일 경우 False를 반환합니다.
    """
    filename = filepath.split('/')[-1]
    return False if filename.startswith('._') else True

def get_img(path):
    """
    이미지를 불러옵니다.
    """
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

class MaskDataset(Dataset):
    def __init__(self, image_dir, info, transform=None):
        self.image_dir = image_dir
        self.info = info
        self.transform = transform
        
        self.image_paths = [path for name in info.path.values for path in glob(os.path.join(image_dir, name, '*'))]
        self.image_paths = list(filter(is_image_file, self.image_paths))
        self.image_paths = list(filter(remove_hidden_file, self.image_paths))
        
        self.labels = [convert_label(path, sep=False) for path in self.image_paths]

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = get_img(image_path)

        if self.transform:
            image = self.transform(image=image)['image']
        label = torch.eye(18)[label]
        return image, label

    def __len__(self):
        return len(self.image_paths)

In [19]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image_path = self.img_paths[index]
        image = get_img(image_path)

        if self.transform:
            image = self.transform(image=image)['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [20]:
import torch
import torch.nn as nn
# import timm

class MyModel(nn.Module):
    def __init__(self, model_name):
        super(MyModel, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=18)
    
    def forward(self, x):
        y_hat = self.model(x)
        return y_hat

In [21]:
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing=0.1, average='mean', log_softmax=True):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.average = average
        self.log_softmax = log_softmax

    def forward(self, x, target):
        x = x.float()
        target = target.float()
        if self.log_softmax: x = x.log_softmax(dim=-1)
        nll_loss = -x * target
        nll_loss = nll_loss.sum(-1)
        smooth_loss = -x.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        if self.average == 'mean':
            return loss.mean()
        else:
            return loss

In [22]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import pytorch_lightning as pl
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.metrics import Accuracy
from adamp import AdamP

class LitTrainer(pl.LightningModule):
    def __init__(self, model_conf):
        super(LitTrainer, self).__init__()
        self.save_hyperparameters('model_conf')
        self.conf = model_conf
        self.model = MyModel(self.conf.model_name)
        self.criterion = LabelSmoothing()
        self.evaluator = Accuracy()
        # Bugfix
        ##################################################
        self.test_result = []
        ##################################################

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
            
        y_hat = self(x)
        train_loss = self.criterion(y_hat, y)
        self.log('train_loss', train_loss)
        
        y = y.argmax(dim=1)
        y_hat = y_hat.argmax(dim=1)
        train_score = self.evaluator(y_hat, y)
        self.log('train_score', train_score)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
            
        y_hat = self(x)
        valid_loss = self.criterion(y_hat, y)
        self.log('valid_loss', valid_loss)
        
        y = y.argmax(dim=1)
        y_hat = y_hat.argmax(dim=1)
        valid_score = self.evaluator(y_hat, y)
        self.log('valid_score', valid_score, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x = batch
        y_hat= self(x)

        score = torch.nn.functional.softmax(y_hat, dim=-1)
        
        y_hat = self(torch.flip(x, dims=[-1]))
        score1 = (score + torch.nn.functional.softmax(y_hat, dim=-1)) / 2.
        return {"score": score}

    def test_epoch_end(self, output_results):
        all_score = torch.cat([out["score"] for out in output_results], dim=0).cpu().numpy()
        # Bugfix
        ##################################################
        self.test_result = {'all_score': all_score}
#         return {'all_score': all_score}
        ##################################################
    
    def configure_optimizers(self):
        if self.conf.optimizer == 'AdamP':
            optimizer = AdamP(self.parameters(), lr=self.conf.learning_rate)
        elif self.conf.optimizer == 'Adam':
            optimizer = Adam(self.parameters(), lr=self.conf.learning_rate)
        elif self.conf.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.conf.learning_rate)
        else:
            raise NotImplementedError(f'{self.conf.optimizer} is not used!')
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=1)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
        }


In [23]:
seed_everything(conf.seed)

info = pd.read_csv(f'{conf.data_dir}/train.csv')
if conf.debug: 
    info = info.iloc[:40]
info.head()

Global seed set to 42


,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [24]:
info['gender_age'] = info.apply(lambda x: convert_gender_age(x.gender, x.age), axis=1)

skf = StratifiedKFold(n_splits=conf.train.n_fold, shuffle=True)
info.loc[:, 'fold'] = 0
for fold_num, (train_index, val_index) in enumerate(skf.split(X=info.index, y=info.gender_age.values)):
    info.loc[info.iloc[val_index].index, 'fold'] = fold_num

In [25]:
image_dir = os.path.join(conf.data_dir, 'images')

for fold_idx in conf.train.use_fold:
    train = info[info.fold != fold_idx].reset_index(drop=True)
    val = info[info.fold == fold_idx].reset_index(drop=True)

    data_conf = conf.dataset
    transforms = get_transforms(img_size=data_conf.image_size)
    train_dataset = MaskDataset(image_dir, train, transforms['train'])
    val_dataset = MaskDataset(image_dir, val, transforms['val'])
    train_loader = DataLoader(train_dataset, batch_size=data_conf.batch_size, shuffle=True, num_workers=data_conf.num_workers)
    val_loader = DataLoader(val_dataset, batch_size=data_conf.batch_size, shuffle=False, num_workers=data_conf.num_workers)
    
    fold_log_dir = os.path.join(conf.log_dir, f'fold{fold_idx}')
    os.makedirs(fold_log_dir, exist_ok=True)
    tb_logger = TensorBoardLogger(save_dir=conf.log_dir, version=f'fold{fold_idx}')
    early_stop_callback = EarlyStopping(monitor='valid_score', mode='max',
                                        patience=conf.train.n_epochs//5, verbose=False)

    checkpoint_callback = ModelCheckpoint(dirpath=fold_log_dir,
                                          filename="{epoch:02d}-{valid_score:.4f}",
                                          monitor='valid_score', mode='max', verbose=False)
    model = LitTrainer(conf.network)
    
    logger = tb_logger if conf.tb else None
    trainer = pl.Trainer(gpus=len(conf.device_list), precision=16,
                         max_epochs=conf.train.n_epochs,
                         progress_bar_refresh_rate=1,
                         logger=logger, callbacks=[checkpoint_callback, early_stop_callback])

    trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | model     | MyModel        | 4.0 M 
1 | criterion | LabelSmoothing | 0     
2 | evaluator | Accuracy       | 0     
---------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.122    Total estimated model params size (MB)


Loaded pretrained weights for efficientnet-b0


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


In [26]:
if conf.run_test:
    submission = pd.read_csv(os.path.join(conf.test_dir, 'info.csv'))
    if conf.debug: 
        submission = submission.iloc[:100]
    image_dir = os.path.join(conf.test_dir, 'images')

    image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
    dataset = TestDataset(image_paths, transforms['val'])

    loader = DataLoader(
        dataset,
        num_workers=0,
        batch_size=48,
        shuffle=False
    )
    
    score = []
    for fold_idx in conf.train.use_fold:
        ckpt_path = glob(os.path.join(conf.log_dir, f'fold{fold_idx}/*.ckpt'))[0]
        model = LitTrainer.load_from_checkpoint(ckpt_path, model_conf=conf.network)
        tester = pl.Trainer(gpus=1, auto_select_gpus=True)
        # Bugfix
        ##################################################
        tester.test(model, loader, verbose=False)[0]
        # prob = tester.test(model, loader, verbose=False)[0]
        score.append(model.test_result['all_score'])
        # score.append(prob['all_score'])
        ##################################################
        
    output = np.mean(score, axis=0).argmax(axis=-1)
    submission['ans'] = output

    submission.to_csv(os.path.join(conf.log_dir, 'submission.csv'), index=False)
    print('test inference is done!')

Loaded pretrained weights for efficientnet-b0


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



test inference is done!
